# Graph Coloring Problem

The **graph coloring problem** can be described mathematically as follows:

Let G = (V, E) be an undirected graph, where:

- V is the set of vertices (or nodes) of the graph.
- E is the set of edges, where each edge e = (u, v) represents a connection between two vertices u and v.

---

### Objective

The goal is to assign a color to each vertex in V such that no two adjacent vertices share the same color. The **chromatic number** χ(G) of the graph G is the minimum number of colors required to color the vertices of the graph in this way. Formally:

χ(G) = min { k : there exists a function f : V → {1, 2, ..., k} such that for all (u, v) ∈ E, f(u) ≠ f(v) }

Where:

- f : V → {1, 2, ..., k} is a coloring function that assigns a color (represented by a number) to each vertex.
- The constraint f(u) ≠ f(v) ensures that adjacent vertices do not share the same color.

---

### Computational Complexity

Determining the chromatic number χ(G) of a graph is a well-known **NP-hard** problem. This means that, in general, there is no efficient algorithm that can find the exact chromatic number for arbitrary graphs in polynomial time, unless P = NP.

Thus, finding the optimal solution (i.e., the exact chromatic number) is computationally expensive, especially for large graphs.

---

### Exact Methods

Despite the NP-hardness, several exact methods can find the optimal solution for moderately sized graphs:

- **Branch and Bound**: This approach systematically explores potential solutions by building a search tree. It uses pruning strategies to eliminate branches that cannot lead to an optimal solution, reducing the search space. The algorithm maintains an upper bound on the chromatic number and prunes branches that would exceed this bound.

- **Dynamic Programming**: For certain classes of graphs or when using specific state representations, dynamic programming can be applied. The method works by storing solutions to overlapping subproblems to avoid redundant computation. For graph coloring, it typically uses a state representation based on subsets of vertices that have been colored so far.

- **Integer Linear Programming (ILP)**: The graph coloring problem can be formulated as an integer linear program where the objective is to minimize the number of colors used, subject to constraints that ensure no adjacent vertices share the same color.

- **Backtracking**: This algorithm explores all possible colorings through depth-first search. For each vertex, it tries each color and backtracks if a valid coloring cannot be found. While simple to implement, basic backtracking can be inefficient for large graphs without additional optimization.

- **Exact Algorithms for Special Graph Classes**: For certain graph classes (like bipartite graphs, perfect graphs, or chordal graphs), specialized algorithms can determine the chromatic number efficiently.

These exact methods guarantee finding the optimal solution but generally have exponential time complexity in the worst case. They are practical for small to medium-sized graphs but become computationally infeasible for large instances.

In [1]:
import time
from typing import List, Dict, Set, Tuple

### Undirected Graph Representation Using an Adjacency List

The `Graph` class models a simple undirected graph using an **adjacency list** representation. This approach is well-suited for **sparse graphs**, where the number of edges is significantly lower than the total number of possible connections.

#### Key characteristics:
- The graph is **undirected**, meaning edges are bidirectional.
- Nodes are indexed as integers from `0` to `n-1`.
- Each node maintains a list of its directly connected neighbors.

#### Main methods:
- `add_edge(u, v)`: Adds an undirected edge between nodes `u` and `v`.
- `get_neighbors(v)`: Returns the list of nodes adjacent to node `v`.
- `get_degree(v)`: Returns the number of neighbors (degree) of node `v`.

In [2]:
class Graph:
    """Représente un graphe simple non-orienté."""
    def __init__(self, num_nodes):
        self.n = num_nodes
        self.adj = [[] for _ in range(num_nodes)]  # Liste d'adjacence

    def add_edge(self, u, v):
        """Ajoute une arête entre u et v."""
        self.adj[u].append(v)
        self.adj[v].append(u)

    def get_neighbors(self, v):
        """Retourne les voisins du sommet v."""
        return self.adj[v]

    def get_degree(self, v):
        """Retourne le degré du sommet v."""
        return len(self.adj[v])
    
    @classmethod
    def from_edges(cls, num_vertices: int, edges: List[Tuple[int, int]]):
        """Create a graph from a list of edges"""
        graph = cls(num_vertices)
        for u, v in edges:
            graph.add_edge(u, v)
        return graph
    
    def __str__(self):
        return f"Graph with {self.n} vertices and adjacency lists:\n{self.adj}"

# Branch and Bound

In [3]:
class BranchAndBoundGraphColoring:
    def __init__(self, graph: Graph):
        self.graph = graph
        self.num_vertices = graph.n
        self.best_coloring = None
        self.min_colors_used = float('inf')
    
    def is_safe(self, vertex: int, color: int, colors: List[int]) -> bool:
        """Check if vertex can be colored with given color"""
        for neighbor in self.graph.get_neighbors(vertex):
            if colors[neighbor] == color:
                return False
        return True
    
    def _graph_coloring_util(self, colors: List[int], vertex: int, max_colors: int) -> bool:
        """Recursive utility function for graph coloring"""
        # If all vertices are colored
        if vertex == self.num_vertices:
            # Count the number of distinct colors used
            colors_used = len(set(colors))
            if colors_used < self.min_colors_used:
                self.min_colors_used = colors_used
                self.best_coloring = colors.copy()
            return True
        
        result = False
        # Try different colors for current vertex
        for color in range(1, max_colors + 1):
            # If exceeds current best solution, prune
            if color > self.min_colors_used:
                break
                
            # Check if color is safe to assign
            if self.is_safe(vertex, color, colors):
                # Assign color
                colors[vertex] = color
                
                # Recur to assign colors to rest of the vertices
                result = self._graph_coloring_util(colors, vertex + 1, max_colors) or result
                
                # Backtrack
                colors[vertex] = 0
        
        return result
    
    def color_graph(self, max_colors=None) -> Tuple[List[int], int]:
        """
        Color the graph using branch and bound
        
        Args:
            max_colors: Maximum number of colors to use (default is number of vertices)
        
        Returns:
            Tuple of (color assignment list, minimum number of colors used)
        """
        if max_colors is None:
            max_colors = self.num_vertices
        
        # Initialize colors list as all zeros (uncolored)
        colors = [0] * self.num_vertices
        
        # Initialize with worst case (every vertex gets a different color)
        self.min_colors_used = self.num_vertices
        self.best_coloring = list(range(1, self.num_vertices + 1))
        
        # Try to color the graph
        self._graph_coloring_util(colors, 0, max_colors)
        
        return self.best_coloring, self.min_colors_used

# Dynamic Programming

In [4]:
class DynamicProgrammingGraphColoring:
    def __init__(self, graph: Graph):
        self.graph = graph
        self.num_vertices = graph.n
        # DP table: dp[mask] = minimum colors needed to color vertices in mask
        self.dp = {}
        # Store coloring: coloring[mask] = dict mapping vertex to color
        self.coloring = {}
    
    def is_independent_set(self, vertices: List[int]) -> bool:
        """Check if a set of vertices forms an independent set (no edges between them)"""
        for i in range(len(vertices)):
            v = vertices[i]
            neighbors = set(self.graph.get_neighbors(v))
            for j in range(i+1, len(vertices)):
                if vertices[j] in neighbors:
                    return False
        return True
    
    def _get_vertices_from_mask(self, mask: int) -> List[int]:
        """Get list of vertices represented by a bit mask"""
        return [i for i in range(self.num_vertices) if (mask & (1 << i))]
    
    def _get_mask_from_vertices(self, vertices: List[int]) -> int:
        """Get bit mask from a list of vertices"""
        mask = 0
        for v in vertices:
            mask |= (1 << v)
        return mask
    
    def color_graph(self) -> Tuple[List[int], int]:
        """
        Color the graph using dynamic programming approach
        
        Returns:
            Tuple of (color assignment list, minimum number of colors used)
        """
        # Initialize DP table
        self.dp = {}
        self.coloring = {}
        
        # Base case: empty set requires 0 colors
        self.dp[0] = 0
        self.coloring[0] = {}
        
        # All possible subsets of vertices (represented as bit masks)
        all_vertices_mask = (1 << self.num_vertices) - 1
        
        # Bottom-up DP
        for mask in range(1, all_vertices_mask + 1):
            self.dp[mask] = float('inf')
            vertices = self._get_vertices_from_mask(mask)
            
            # Try all possible independent sets as one color class
            for submask in range(1, 1 << len(vertices)):
                color_class = [vertices[i] for i in range(len(vertices)) if submask & (1 << i)]
                
                # Check if this forms an independent set
                if self.is_independent_set(color_class):
                    # Remove these vertices and recur
                    remaining_mask = mask ^ self._get_mask_from_vertices(color_class)
                    
                    if 1 + self.dp[remaining_mask] < self.dp[mask]:
                        self.dp[mask] = 1 + self.dp[remaining_mask]
                        
                        # Update coloring
                        self.coloring[mask] = self.coloring[remaining_mask].copy()
                        color = self.dp[mask]  # Use the DP value as the color
                        for v in color_class:
                            self.coloring[mask][v] = color
        
        # Reconstruct the coloring
        final_coloring = [0] * self.num_vertices
        for v, color in self.coloring[all_vertices_mask].items():
            final_coloring[v] = color
        
        # Remap colors to start from 1 and be consecutive
        color_map = {}
        next_color = 1
        for i in range(self.num_vertices):
            if final_coloring[i] not in color_map:
                color_map[final_coloring[i]] = next_color
                next_color += 1
            final_coloring[i] = color_map[final_coloring[i]]
        
        return final_coloring, self.dp[all_vertices_mask]

In [5]:
def generate_test_graphs():
    """Generate multiple test graphs for evaluation"""
    graphs = []
    
    # Test Graph 1: Small cycle graph
    cycle_graph = Graph.from_edges(5, [(0, 1), (1, 2), (2, 3), (3, 4), (4, 0)])
    graphs.append(("Cycle Graph (5 vertices)", cycle_graph))
    
    # Test Graph 2: Complete graph (K4)
    complete_graph = Graph.from_edges(4, [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)])
    graphs.append(("Complete Graph K4", complete_graph))
    
    # Test Graph 3: Petersen graph
    petersen_edges = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 0),  # Outer pentagon
                      (0, 5), (1, 6), (2, 7), (3, 8), (4, 9),  # Spokes
                      (5, 7), (7, 9), (9, 6), (6, 8), (8, 5)]   # Inner pentagram
    petersen_graph = Graph.from_edges(10, petersen_edges)
    graphs.append(("Petersen Graph", petersen_graph))
    
    # Test Graph 4: A custom graph
    custom_edges = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4), (2, 4)]
    custom_graph = Graph.from_edges(5, custom_edges)
    graphs.append(("Custom Graph", custom_graph))
    
    # Test Graph 5: Bipartite graph
    bipartite_edges = [(0, 3), (0, 4), (0, 5), 
                       (1, 3), (1, 4), (1, 5),
                       (2, 3), (2, 4), (2, 5)]
    bipartite_graph = Graph.from_edges(6, bipartite_edges)
    graphs.append(("Bipartite Graph", bipartite_graph))

    # Test Graph 6: Grid graph (3x3)
    grid_edges = [(0, 1), (1, 2), (3, 4), (4, 5), (6, 7), (7, 8),  # Horizontal edges
                 (0, 3), (3, 6), (1, 4), (4, 7), (2, 5), (5, 8)]  # Vertical edges
    grid_graph = Graph.from_edges(9, grid_edges)
    graphs.append(("Grid Graph (3x3)", grid_graph))
    
    return graphs

In [6]:
def verify_coloring(graph: Graph, coloring: List[int]) -> bool:
    """Verify that a coloring is valid (no adjacent vertices have same color)"""
    for v in range(graph.n):
        for neighbor in graph.get_neighbors(v):
            if coloring[v] == coloring[neighbor]:
                return False
    return True

# Tests

In [7]:

def test_algorithms():
    """Test both algorithms on multiple graphs and compare results"""
    graphs = generate_test_graphs()
    
    print("Testing Graph Coloring Algorithms")
    print("=" * 50)
    
    for name, graph in graphs:
        print(f"\nGraph: {name}")
        print("-" * 40)
        
        # Test Branch and Bound
        start_time = time.time()
        bb_solver = BranchAndBoundGraphColoring(graph)
        bb_coloring, bb_colors = bb_solver.color_graph()
        bb_time = time.time() - start_time
        
        print(f"Branch and Bound Result:")
        print(f"  Colors used: {bb_colors}")
        print(f"  Coloring: {bb_coloring}")
        print(f"  Time: {bb_time:.6f} seconds")
        
        # Test Dynamic Programming (only for small graphs due to exponential memory)
        if graph.n <= 12:  # DP approach is memory intensive
            start_time = time.time()
            dp_solver = DynamicProgrammingGraphColoring(graph)
            dp_coloring, dp_colors = dp_solver.color_graph()
            dp_time = time.time() - start_time
            
            print(f"Dynamic Programming Result:")
            print(f"  Colors used: {dp_colors}")
            print(f"  Coloring: {dp_coloring}")
            print(f"  Time: {dp_time:.6f} seconds")
        else:
            print("Dynamic Programming skipped (graph too large)")
        
        # Verify the solutions
        is_valid_bb = verify_coloring(graph, bb_coloring)
        print(f"Branch and Bound solution valid: {is_valid_bb}")
        
        if graph.n <= 12:
            is_valid_dp = verify_coloring(graph, dp_coloring)
            print(f"Dynamic Programming solution valid: {is_valid_dp}")

if __name__ == "__main__":
    test_algorithms()

Testing Graph Coloring Algorithms

Graph: Cycle Graph (5 vertices)
----------------------------------------
Branch and Bound Result:
  Colors used: 3
  Coloring: [1, 2, 1, 2, 3]
  Time: 0.000082 seconds
Dynamic Programming Result:
  Colors used: 3
  Coloring: [1, 2, 3, 2, 3]
  Time: 0.000399 seconds
Branch and Bound solution valid: True
Dynamic Programming solution valid: True

Graph: Complete Graph K4
----------------------------------------
Branch and Bound Result:
  Colors used: 4
  Coloring: [1, 2, 3, 4]
  Time: 0.000061 seconds
Dynamic Programming Result:
  Colors used: 4
  Coloring: [1, 2, 3, 4]
  Time: 0.000119 seconds
Branch and Bound solution valid: True
Dynamic Programming solution valid: True

Graph: Petersen Graph
----------------------------------------
Branch and Bound Result:
  Colors used: 3
  Coloring: [1, 2, 1, 2, 3, 2, 1, 3, 3, 2]
  Time: 0.000560 seconds
Dynamic Programming Result:
  Colors used: 3
  Coloring: [1, 2, 1, 2, 3, 2, 3, 3, 1, 1]
  Time: 0.107308 seconds
